# Explore Fetchcore API

In [ ]:
import os
import requests
import json

In [ ]:
# Get AUTH HEADERS
with open(os.path.expanduser('~/sup_ws/fetchcore_token.txt')) as f:
    token=f.read().strip()
headers = {'Authorization' : 'Bearer {}'.format(token)}

In [ ]:
customer='zebra-support'
url='https://{}.fetchcore-cloud.com'.format(customer)

## Simple Examples

### Get all api v1 endpoints

In [ ]:
api_v1=url+'/api/v1'
r=requests.get(api_v1,headers=headers)
resp=r.json()

### Get System Information

In [ ]:
system=url+'/api/v1/system'
r=requests.get(system,headers=headers)
resp=r.json()
print(resp)

## Scheduler

### Get all schedules

In [ ]:
get_all_schedules=url+'/api/v1/scheduler/workflow-event?calendar=1&offset=0&limit=200'
r=requests.get(get_all_schedules,headers=headers)
resp=r.json()

# Print the first schedule
s=resp['results'][0]['start_event_args']
print(json.dumps(s,indent=2))

### Create a new schedule

In [ ]:
import uuid
from datetime import datetime,timedelta
from pytz import timezone

# Send a POST request to this API endpoint
create_schedule=url+'/api/v1/scheduler/workflow-event'

TZ='Asia/Kuala_Lumpur'
now=datetime.now(tz=timezone(TZ))
f='%Y%m%dT%H%M%S'
start=datetime.strftime(now,f)
end=datetime.strftime(now+timedelta(hours=1),f)

# Workflow information 1 Workflow 1 action Group
workflow='ccd493be-e176-4d3b-98f3-0aa41e02788e'
block_group='e7bc494c-46de-4391-b666-14510c118263'
robot='freight100-1737'

# UUID needed in the payload
u=str(uuid.uuid4())

# Repeat Rule (1 time occurrence or repeated weekly)
repeat_rule='BYDAY=SU,FR,SA,MO'
#repeat_rule='COUNT=1'

# UNTIL
until=''


# Start and end rrule
start_rrule='DTSTART;TZID={tz}:{t}\nRRULE:FREQ=WEEKLY;INTERVAL=1;WKST=MO;{repeat_rule}'.format(
    tz=TZ,t=start,repeat_rule=repeat_rule
)
end_rrule='DTSTART;TZID={tz}:{t}\nRRULE:FREQ=WEEKLY;INTERVAL=1;WKST=MO;{repeat_rule}'.format(
    tz=TZ,t=end,repeat_rule=repeat_rule
)

# start and event_args is the same during schedule creation 
event_args=[
    {
        'id':u,
        'workflow':workflow,
        'resources':[
            {
                'type':'robot',
                'name':robot
            }
        ],
        'block_group':block_group
    }         
]
event_args=json.dumps(event_args)

# This is the JSON data needed in the payload of the POST request
data={
    'calendar':1,
    'end_event_args':event_args,
    'end_event_task':'trigger_workflow_stop@1.0.0',
    'name':'',
    'end_rrule':end_rrule,
    'start_event_args':event_args,
    'start_event_task':'trigger_workflow_start@1.0.0',
    'start_rrule':start_rrule
}
# SUMMARY
print('-'*50)
print('POST to API endpoint:',create_schedule)
print('with the Payload: ')
print(json.dumps(data,indent=2))
r=requests.post(create_schedule,json=data,headers=headers)
print('-'*50)
print('Status code:',r.status_code)
print('Response: ')
print(r.json())